Exercício I2A2 - K-Means/K-NN - "Catch Me If You Can" dataset (https://www.kaggle.com/danielkurniadi/catch-me-if-you-can)


O exercício envolve a utilização dos dados de teste do dataset "Catch Me If You Can" (test_sessions.csv) 
e a aplicação do da metodologia K-Means sobre os dados de teste para clusterização com 2 K's (clusters) com a finalidade de identificar
o target Alice (0 ou 1, conforme indetificado nos dados de treino).

Após a aplicação do K-Means, treina-se a metodlogia o K-Nearest Neighbors sobre os dados de treino e aplica-se sobre os mesmos dados de treino.

Compara-se com os resultados obtidos com o K-Means.


In [0]:
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


In [0]:
train_df = pd.read_csv('train_sessions.csv')
data_train_df = train_df.dropna()

test_df = pd.read_csv('test_sessions.csv')
data_test_df = test_df.dropna()


Para fins da aplicação do K-Means e do K-NN optou-aw por não utilizar os dados temporais visando analisar apenas os dados relativos aos sites visitados.
Adicionalmente, somente foram analisado os casos em que houve 10 (visitados) sites visitados.
Os sites visitados são identificados com números únicos, de forma que cada site possui uma numeração, e salvo engano, podem ser representado, para fins destas metodologias pelos números que lhe são atribuídos ocupando uma localização única dentro do espaço analisado. 

Inicialmente, realizou-se o tratamento dos dados de teste para fins da aplicação da metodologia K-Means.


In [0]:
times = ['time%s' % i for i in range(1, 11)]


In [0]:
km_data_test_df = data_test_df.drop(times,axis=1)
km_data_test_df = km_data_test_df.drop('session_id',axis=1)


In [5]:
kmeans = KMeans(n_clusters=2,random_state=0)
kmeans.fit(km_data_test_df)


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)

In [0]:
km_labels = kmeans.labels_


In [7]:
print('{0} e {1} instâncias foram classificados nos clusters 0 e 1, respectivamente, do total de {2} instâncias'.format(len(km_labels)-sum(km_labels), sum(km_labels), len(km_labels)))


66546 e 8260 instâncias foram classificados nos clusters 0 e 1, respectivamente, do total de 74806 instâncias


In [0]:
km_data_test_df_final = pd.concat([km_data_test_df,  pd.DataFrame(km_labels)], axis=1)


In [0]:
km_data_test_df_final.to_csv('kmeans_result.csv')


Realizou-se o tratamento dos dados de treino para fins da aplicação da metodologia K-NN.

In [0]:
knn_data_train_df_stimes = data_train_df.drop(times,axis=1)
knn_data_train_df = knn_data_train_df_stimes.drop(['session_id', 'target'],axis=1)
knn_data_train_df_target = knn_data_train_df_stimes['target']


In [11]:
knn = KNeighborsClassifier(n_neighbors=6)
knn.fit(knn_data_train_df, knn_data_train_df_target)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=6, p=2,
           weights='uniform')

In [0]:
knn_target = knn.predict(km_data_test_df)


In [15]:
print('{0} e {1} instâncias foram classificados nos clusters 0 e 1, respectivamente, do total de {2} instâncias'.format(len(knn_target)-sum(knn_target), sum(knn_target), len(knn_target)))


74803 e 3 instâncias foram classificados nos clusters 0 e 1, respectivamente, do total de 74806 instâncias


In [0]:
knn_data_test_df_final = pd.concat([km_data_test_df,  pd.DataFrame(knn_target)], axis=1)


In [0]:
knn_data_test_df_final.to_csv('knn_result.csv')


Realizamos uma simples comparação entres as duas metodologias, tendo em vista que para o K-Means K-NN obtivemos  as instâncias foram classificados nos clusters 0 e 1.
A classificação nos clusters 0 e 1 é realizada de forma arbitrária pelos algoritmos, de forma que para fins de comparação pode se fazer necessária a troca inversão da classificação, para que ambos o resultado reflitam as mesmas categorias. Não obstante, em razão dos resultados obtidos entendemos que não é necessária a realização da inversão.


In [20]:
comparison = knn_target == km_labels
print('{0}% dos resultados são consistentes entre as metodologias'.format(sum(comparison)/len(comparison)*100))


88.95409459134294% dos resultados são consistentes entre as metodologias


Para fins da escolha do K do K-NN utilizamos a iteração abaixo.


In [22]:
from sklearn.model_selection import train_test_split
knn_train, knn_test = train_test_split(knn_data_train_df_stimes, test_size=0.2, random_state=10)
i = 1
knn_training = knn_train.drop(['session_id', 'target'],axis=1)
knn_training_target = knn_train['target']

knn_testing = knn_test.drop(['session_id', 'target'],axis=1)
knn_testing_target = knn_test['target']

while i <= 10:
  
  k_testing = KNeighborsClassifier(n_neighbors=i)
  k_testing.fit(knn_training, knn_training_target)
  
  k_score = k_testing.score(knn_testing, knn_testing_target, sample_weight=None)
  print('k = {0}, score = {1}'.format(i, k_score))
  
  i += 1
  

k = 1, score = 0.9825366254787821
k = 2, score = 0.9898725411698513
k = 3, score = 0.9891584254831101
k = 4, score = 0.9901971392092792
k = 5, score = 0.9899591006470321
k = 6, score = 0.9903269784250504
k = 7, score = 0.9902187790785744
k = 8, score = 0.9902620588171648
k = 9, score = 0.9902404189478695
k = 10, score = 0.9903269784250504
